# lasagne
* lasagne is a library for neural network building and training
* it's a low-level library with almost seamless integration with theano

For a demo we shall solve the same digit recognition problem, but at a different scale
* images are now 28x28
* 10 different digits
* 50k samples

In [312]:
import numpy as np
import theano
import theano.tensor as T
import matplotlib.pyplot as plt
%matplotlib inline

In [313]:
from mnist import load_dataset
X_train,y_train,X_val,y_val,X_test,y_test = load_dataset()

print(X_train.shape,y_train.shape, y_val)

(50000, 1, 28, 28) (50000,) [3 8 6 ..., 5 6 8]


In [314]:
input_X = T.tensor4("X")

#input dimention (None means "Arbitrary" and only works at  the first axes [samples])
input_shape = [None,1,28,28]

target_y = T.vector("target Y integer",dtype='int32')

Defining network architecture

In [315]:
import lasagne
from lasagne.layers import *

#Input layer (auxilary)
l1 = InputLayer(shape = input_shape,input_var=input_X)

#fully connected layer, that takes input layer and applies 50 neurons to it.
# nonlinearity here is sigmoid as in logistic regression
# you can give a name to each layer (optional)
l = lasagne.layers.Conv2DLayer(l1, num_filters=20, filter_size=5, nonlinearity=lasagne.nonlinearities.linear)
l = lasagne.layers.MaxPool2DLayer(l, pool_size=(2,2))
l = lasagne.layers.Conv2DLayer(l, num_filters=10, filter_size=5, nonlinearity=lasagne.nonlinearities.linear)
l = lasagne.layers.MaxPool2DLayer(l, pool_size=(2,2))
l = lasagne.layers.DropoutLayer(l,  p=0.6)
l = lasagne.layers.DenseLayer(l, num_units=1000, nonlinearity=lasagne.nonlinearities.tanh)

#Fully-connected + dropout

#l_2D = lasagne.layers.Conv2DLayer(l1,5,3) 
#l2 = DenseLayer(l_2D,num_units=50,nonlinearity = lasagne.nonlinearities.elu)

# Another 50-unit layer:
l_out = DenseLayer(l,num_units = 10, nonlinearity=lasagne.nonlinearities.softmax)


In [316]:
#network prediction (theano-transformation)
y_predicted = lasagne.layers.get_output(l_out)
y_predicted_test = lasagne.layers.get_output(l_out, deterministic = True)

In [317]:
#all network weights (shared variables)
all_weights = lasagne.layers.get_all_params(l_out)
print (all_weights)

[W, b, W, b, W, b, W, b]


### Than you could simply
* define loss function manually
* compute error gradient over all weights
* define updates
* But that's a whole lot of work and life's short
  * not to mention life's too short to wait for SGD to converge

Instead, we shall use Lasagne builtins

In [318]:
#Mean categorical crossentropy as a loss function - similar to logistic loss but for multiclass targets
loss = lasagne.objectives.categorical_crossentropy(y_predicted,target_y).mean()

#prediction accuracy
accuracy = lasagne.objectives.categorical_accuracy(y_predicted_test,target_y).mean()

#This function computes gradient AND composes weight updates just like you did earlier
updates_sgd = lasagne.updates.adadelta(loss, all_weights,learning_rate=0.3)

In [319]:
#function that computes loss and updates weights
train_fun = theano.function([input_X,target_y],[loss,accuracy],updates= updates_sgd)

#function that just computes accuracy
accuracy_fun = theano.function([input_X,target_y],accuracy)

### That's all, now let's train it!
* We got a lot of data, so it's recommended that you use SGD
* So let's implement a function that splits the training sample into minibatches

In [320]:
# An auxilary function that returns mini-batches for neural network training

#Parameters
# inputs - a tensor of images with shape (many, 1, 28, 28), e.g. X_train
# outputs - a vector of answers for corresponding images e.g. Y_train
#batch_size - a single number - the intended size of each batches

def iterate_minibatches(inputs, targets, batchsize):
    assert len(inputs) == len(targets)
    indices = np.arange(len(inputs))
    np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        excerpt = indices[start_idx:start_idx + batchsize]
        yield inputs[excerpt], targets[excerpt]

# Training loop

In [321]:
import time

num_epochs = 100 #amount of passes through the data
l=[]
batch_size = 50 #number of samples processed at each function call
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train,batch_size):
        inputs, targets = batch
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))

    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(
        train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))
    #l.append(train_err / train_batches)
    #plt.plot(list(range(epoch+1)),l,'r')
   

Epoch 1 of 100 took 65.816s
  training loss (in-iteration):		0.387941
  train accuracy:		92.70 %
  validation accuracy:		96.34 %
Epoch 2 of 100 took 69.653s
  training loss (in-iteration):		0.228149
  train accuracy:		96.26 %
  validation accuracy:		97.08 %
Epoch 3 of 100 took 71.643s
  training loss (in-iteration):		0.193594
  train accuracy:		96.99 %
  validation accuracy:		96.88 %
Epoch 4 of 100 took 79.422s
  training loss (in-iteration):		0.172524
  train accuracy:		97.39 %
  validation accuracy:		97.76 %
Epoch 5 of 100 took 74.623s
  training loss (in-iteration):		0.152024
  train accuracy:		97.74 %
  validation accuracy:		98.18 %
Epoch 6 of 100 took 73.651s
  training loss (in-iteration):		0.140545
  train accuracy:		97.92 %
  validation accuracy:		98.25 %
Epoch 7 of 100 took 69.681s
  training loss (in-iteration):		0.126300
  train accuracy:		98.28 %
  validation accuracy:		98.37 %
Epoch 8 of 100 took 73.006s
  training loss (in-iteration):		0.115886
  train accuracy:		98.45 %


KeyboardInterrupt: 

In [322]:
plt.show()
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 99:
    print ("Achievement unlocked: 80lvl Warlock!")
else:
    print ("We need more magic!")

Final results:
  test accuracy:		99.14 %
Achievement unlocked: 80lvl Warlock!


In [ ]:
y = arccos( (( 10^4 + 3726.9 )*10*8 / 10^4 )*sqrt(( (1 /(0.05*sqrt(2* 3.14 ) ) )*exp( -0.5((x - 2)/0.05)^2  ))/(2*3726.9 +(1 /(0.05*sqrt(2* 3.14 ) ) )*exp( -0.5((x - 2)/0.05)^2 )) )) 

# Now improve it!

* Moar layers!
* Moar units!
* Different nonlinearities!